# Tech Challenge

## Descrição

Você foi contratado(a) para uma consultoria e seu trabalho envolve analisar os dados de vitivinicultura da Embrapa, os quais estão disponíveis [aqui](http://vitibrasil.cnpuv.embrapa.br/index.php).

A ideia do projeto é a criação de uma _API_ pública de consulta nos dados do site nas respectivas abas:

* Produção
* Processamento
* Comercialização
* Importação
* Exportação

A _API_ vai servir para alimentar uma base de dados que futuramente será usada para um modelo de _Machine Learning_.

Seus objetivos incluem:

* Criar uma Rest _API_ em [Python](https://www.python.org/) que faça a consulta no [site da Embrapa](http://vitibrasil.cnpuv.embrapa.br/index.php).
* Sua _API_ deve estar documentada.
* É recomendável (não obrigatório) a escolha de um método de autenticação ([`JWT`](https://jwt.io), por exemplo).
* Criar um plano para fazer o _deploy_ da _API_, desenhando a arquitetura do projeto desde a ingestão até a alimentação do modelo (aqui não é necessário elaborar um modelo de ML, mas é preciso que vocês escolham um cenário interessante em que a _API_ possa ser utilizada).
* Fazer um _MVP_ realizando o _deploy_ com um link compartilhável e um repositório no [GitHub](https://github.com/LucasVmigotto/fiap-pos-ml-eng-techchallenge-1).

## Setup

### Importação de Libs

In [2]:
from os.path import exists
from os import listdir
from urllib.request import urlretrieve
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn
from tqdm import tqdm
from matplotlib import pyplot as plt

### Constantes

In [3]:
BASE_URL = 'http://vitibrasil.cnpuv.embrapa.br/download/'

CSV_URLS = dict(
    producao=dict(producao='Producao.csv'),
    processamento=dict(viniferas='ProcessaViniferas.csv',
                       americanas='ProcessaAmericanas.csv',
                       mesa='ProcessaMesa.csv',
                       sem_classificacao='ProcessaSemclass.csv'),
    comercializacao=dict(comercio='Comercio.csv'),
    importacao=dict(mesa='ImpVinhos.csv',
                    espumantes='ImpEspumantes.csv',
                    frescas='ImpFrescas.csv',
                    passas='ImpPassas.csv',
                    suco='ImpSuco.csv'),
    exportacao=dict(mesa='ExpVinho.csv',
                    espumantes='ExpEspumantes.csv',
                    frescas='ExpUva.csv',
                    suco='ExpSuco.csv'),
)

In [4]:
PATH_BASE_DATA = Path('./data')
PATH_RAW_DATA = Path('./data') / 'raw'
PATH_INTERIM_DATA = Path('./data') / 'interim'

SEED = 42

np.random.seed(SEED)

PATH_RAW_DATA.mkdir(parents=True, exist_ok=True)
PATH_INTERIM_DATA.mkdir(parents=True, exist_ok=True)

### Utils

In [4]:
def download_files(url: str, save_path: Path, is_retry: bool = False) -> None:
    try:
        if not exists(save_path):
            urlretrieve(url, save_path)
    except ConnectionResetError:
        print(f'File {url} could not be downloaded')
        if not is_retry:
            download_files(url, save_path, True)
        return

In [5]:
def download_data_files(data_files_structure: dict, parent_path: Path = Path('')):
    for key, value in tqdm(data_files_structure.items()):
        if isinstance(value, dict):
            download_data_files(value, parent_path / key)
            continue
        (PATH_RAW_DATA / parent_path).mkdir(parents=True,
                                            exist_ok=True)
        download_files(url=BASE_URL + value,
                       save_path=PATH_RAW_DATA / parent_path / value.lower())

## Coleta dos Dados

In [6]:
download_data_files(CSV_URLS)
listdir(PATH_RAW_DATA)

100%|██████████| 5/5 [00:04<00:00,  1.07it/s]


['producao', 'importacao', 'exportacao', 'comercializacao', 'processamento']

* Verificar o header de cada arquivo `.csv` - conferir o tipo de separador utilizado

In [7]:
! head -n 1 ./data/raw/**/*.csv

==> ./data/raw/comercializacao/comercio.csv <==
1;VINHO DE MESA;VINHO DE MESA;98327606;114399031;118377367;116617910;94173324;108031792;139238614;140813114;141293379;149609112;122825298;128894580;166861772;195616620;171619507;185191837;203130018;131065191;150678647;172921267;164725646;190134895;180230431;201168480;180295366;146583828;165831436;174768638;181576649;200578746;221023603;221518224;227447392;217082959;225021830;271248493;245625614;226710045;200488612;234525979;221242945;230310468;206969571;221590810;206404427;209198468;166769622;176059959;177186273;180446489;215557931;210012238;187939996

==> ./data/raw/exportacao/expespumantes.csv <==
Id;País;1970;1970;1971;1971;1972;1972;1973;1973;1974;1974;1975;1975;1976;1976;1977;1977;1978;1978;1979;1979;1980;1980;1981;1981;1982;1982;1983;1983;1984;1984;1985;1985;1986;1986;1987;1987;1988;1988;1989;1989;1990;1990;1991;1991;1992;1992;1993;1993;1994;1994;1995;1995;1996;1996;1997;1997;1998;1998;1999;1999;2000;2000;2001;2001;2002;2002;2003;20

**Um pouco de cultura**

* "Imaginemos, por um instante, que a humanidade fosse transportada a um país utópico, onde os pombos voem já assados, onde todo o alimento cresça do solo espontaneamente, onde cada homem encontre sua amada ideal e a conquiste sem qualquer dificuldade. Ora, nesse país, muitos homens morreriam de tédio ou se enforcariam nos galhos das árvores, enquanto outros se dedicariam a lutar entre si e a se estrangular, a se assassinar uns aos outros." (Arthur Schopenhauer)

**TL;DR**

* Se fácil, teria graça? (Provavelmente ainda sim, mas onde está o significado da vida sem o estresse?)

**Whatever**

* Todos, com exceção de `processamento` que usa `\t`, usam, como separador, o `;`

## Manipulação dos Dados

In [68]:
dfs = dict.fromkeys(listdir(PATH_RAW_DATA), {})

for category in listdir(PATH_RAW_DATA):
    for data_file in listdir(PATH_RAW_DATA / category):
        dfs[category] = {
            **dfs[category], 
            data_file[:-4]: pd.read_csv(PATH_RAW_DATA / category / data_file,
                                        sep=';' if category != 'processamento' else '\t').iloc[:, 1:]
        }
        
dfs.keys()

dict_keys(['producao', 'importacao', 'exportacao', 'comercializacao', 'processamento'])

In [70]:
def print_head(df_dict: dict, parent_key: str | None = ''):
    for key, value in df_dict.items():
        if isinstance(value, dict):
            print_head(value, key)
        else:
            print(parent_key, key)
            display(value.head())

print_head(dfs)

producao producao


,produto,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,195359778,200053669,...,196904222,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611
1,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,154801826,162917363,...,163111797,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214
2,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,4954387,5079748,...,32066403,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430
3,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,35603565,32056558,...,1726022,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968
4,VINHO FINO DE MESA (VINÍFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,41264971,36750933,...,45782530,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796


importacao impvinhos


,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,1974,...,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1
0,Africa do Sul,0,0.0,0,0,0,0,0,0,0,...,1127053,3574371,1092042,3604038,627150,1701072,859169,2508140,738116.0,2266827.0
1,Alemanha,52297,30498.0,34606,26027,134438,92103,111523,98638,219173,...,142971,516975,101055,412794,136992,504168,106541,546967,92600.0,438595.0
2,Argélia,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
3,Arábia Saudita,0,0.0,0,0,0,0,0,0,0,...,563,3249,0,0,0,0,2510,8761,0.0,0.0
4,Argentina,19525,12260.0,24942,15022,104906,58137,116887,76121,215930,...,15221318,52817642,16548931,54527380,22610267,66322932,26869241,79527959,27980574.0,87519642.0


importacao impfrescas


...

,control,cultivar,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,TINTAS,TINTAS,10448228,11012833,10798824,8213674,17457849,22593885,20265190,24830345,...,36855419,29810706,29935627,13370866,32850915,26868514,nd,28003505,93296587,*
1,ti_Alicante Bouschet,Alicante Bouschet,0,0,0,0,0,0,0,0,...,1524728,1456305,1519576,908841,2040198,2103844,nd,2272985,811140,*
2,ti_Ancelota,Ancelota,0,0,0,0,0,0,0,0,...,1137943,937844,773526,179028,733907,492106,nd,481402,6513974,*
3,ti_Aramon,Aramon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,*
4,ti_Alfrocheiro,Alfrocheiro,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,*


processamento processaamericanas


,control,cultivar,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,TINTAS,TINTAS,284285642,193695427,187251954,161593877,295404693,282003228,222289729,272772106,...,460738996,471555903,537592720,235356261,600605876,519506589,nd,376136313,394224543,*
1,ti_Bacarina,Bacarina,82899,106962,67464,58690,138158,101454,57297,0,...,2990,3900,0,0,0,0,nd,0,0,*
2,ti_Bailey,Bailey,0,0,0,0,0,0,0,0,...,756000,991449,963159,442784,1370092,539742,nd,534981,4092669,*
3,ti_Bordo,Bordo,7242197,7227090,6530686,5584243,13341412,16023998,12725233,18714617,...,102788361,113008320,137467196,60976531,160146475,158405972,nd,129978861,117655879,*
4,ti_Bourdin (S),Bourdin (S),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,*


processamento processasemclass


,control,cultivar,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,sc,Sem classificação,3675463,665425,197232,491357,57307,540146,24440,9743,...,0,0,0,0,0,0,nd,166947,0,*


## Exploração dos Dados